# Demo: Multi-View 6D State Visualization

Purpose:
- Generate a 6D state trajectory (position, velocity, acceleration).
- Visualize it across multiple canvases: scatter, line, heatmap, and polyline graphs.
- Demonstrate linking between canvases and live animation updates.
- Uses `%gui qt5` to integrate Qt into Jupyter (no explicit `app.exec()`).

In [ ]:
%gui qt5

In [ ]:
# Imports and project path setup
import os, sys, time, threading
import numpy as np
from PyQt5 import QtWidgets, QtCore

PROJECT_ROOT = os.path.abspath(os.path.join(os.path.dirname("."), "../.."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from MCVGraph import DataSource, GraphBus, Canvas, ScatterPlot, LinePlot, HeatmapPlot, PolylinePlot
from MCVGraph.widgets.GraphWidget import SNAP_BELOW

In [ ]:
# Data generation: 6D state trajectory
def generate_6d_data(n=200, t=None):
    if t is None:
        t = np.linspace(0, 4 * np.pi, n)
    x = np.sin(t) * (1 + 0.5 * np.cos(2 * t))
    y = np.sin(2 * t) * 0.5
    dx = np.cos(t) * (1 + 0.5 * np.cos(2 * t)) - np.sin(t) * np.sin(2 * t)
    dy = 2 * np.cos(2 * t) * 0.5
    ax = -np.sin(t) * (1 + 0.5 * np.cos(2 * t)) - 2 * np.sin(2 * t) * 0.5
    ay = -4 * np.sin(2 * t) * 0.5
    return np.column_stack((x, y, dx, dy, ax, ay))

In [ ]:
# Transform functions
def pos_transform(data):
    return data[:, 0:2]

def vel_transform(data):
    return data[:, 2:4]

def acc_transform(data):
    return data[:, 4:6]

def density_transform(data, size=64):
    xy = data[:, 0:2]
    nx = (xy[:, 0] + 2.0) / 4.0
    ny = (xy[:, 1] + 1.0) / 2.0
    ix = np.clip((nx * size).astype(int), 0, size - 1)
    iy = np.clip((ny * size).astype(int), 0, size - 1)
    mat = np.zeros((size, size), dtype=np.float32)
    np.add.at(mat, (ix, iy), 1.0)
    return mat

def log_normalizer(arr):
    log_arr = np.log1p(arr)
    vmin, vmax = np.min(log_arr), np.max(log_arr)
    return (log_arr - vmin) / (vmax - vmin) if vmax > vmin else np.zeros_like(arr)

In [ ]:
# Animation loop (runs in background thread)
def run_animation_loop(data_source, edges_ds, *, dt=0.016, stop_event=None):
    t = 0.0
    while stop_event is None or not stop_event.is_set():
        time_vector = np.linspace(t, t + 4 * np.pi, 200)
        state = generate_6d_data(n=200, t=time_vector)
        data_source.set(state)
        pos = state[:, 0:2]
        diff = pos[None, :, :] - pos[:, None, :]
        dist2 = np.sum(diff * diff, axis=2)
        np.fill_diagonal(dist2, np.inf)
        dist = np.sqrt(dist2)
        nearest = np.argsort(dist, axis=1)[:, 1:4]
        edges = []
        for i in range(len(state)):
            for j in nearest[i]:
                edges.append([i, j])
        edges_ds.set(np.array(edges, dtype=int))
        t += dt * 0.1
        time.sleep(dt)

In [ ]:
# Initialize data sources
initial_state = generate_6d_data(n=200)
data_source = DataSource(initial_state)
edges_ds = DataSource(np.zeros((0, 2), dtype=int))

In [ ]:
# Initialize Qt app (managed by %gui qt5)
app = QtWidgets.QApplication.instance() or QtWidgets.QApplication([])

In [ ]:
# Create canvases
canvas_position = Canvas(); canvas_position.setWindowTitle("Position & Velocity Views")
canvas_acceleration = Canvas(); canvas_acceleration.setWindowTitle("Acceleration View")
canvas_sonify = Canvas(); canvas_sonify.setWindowTitle("Sonification: Acceleration (Y)")
canvas_density = Canvas(); canvas_density.setWindowTitle("Density (Heatmap)")
canvas_polyline = Canvas(); canvas_polyline.setWindowTitle("Network (3-NN)")

In [ ]:
# Create graphs
scatter_pos = ScatterPlot(data_source, name="Position"); scatter_pos.set_transform(pos_transform)
scatter_vel = ScatterPlot(data_source, name="Velocity"); scatter_vel.set_transform(vel_transform)
scatter_acc = ScatterPlot(data_source, name="Acceleration"); scatter_acc.set_transform(acc_transform)
line_sonify = LinePlot(data_source, sample_rate=1000, transform=lambda d: d[:, 5], name="AccelY")
heatmap_plot = HeatmapPlot(data_source,
    transform=lambda d: density_transform(d, 64),
    normalizer=log_normalizer,
    scale_x=4.0/64.0, scale_y=2.0/64.0,
    graph_name="Density")
heatmap_plot.set_translation(-2.0, -1.0)
polyline_plot = PolylinePlot(vertices=data_source, edges=edges_ds, color="orange", line_width=1.5)

In [ ]:
# Plot graphs on canvases
canvas_position.plot(scatter_pos)
canvas_position.plot(scatter_vel)
canvas_position.plot(heatmap_plot)
canvas_position.plot(polyline_plot)
canvas_position.show()

canvas_acceleration.plot(scatter_acc); canvas_acceleration.show()
canvas_sonify.plot(line_sonify); canvas_sonify.show()
canvas_density.plot(heatmap_plot); canvas_density.show()
canvas_polyline.plot(polyline_plot); canvas_polyline.show()

In [ ]:
# Example linking of canvases
canvas_position.snap(canvas_acceleration, SNAP_BELOW)
canvas_position.link_target = canvas_acceleration
canvas_position.toggle_auto_match()

In [ ]:
# Start animation in background thread
stop_event = threading.Event()
anim_thread = threading.Thread(
    target=run_animation_loop,
    kwargs=dict(data_source=data_source, edges_ds=edges_ds, dt=0.016, stop_event=stop_event),
    daemon=True)
anim_thread.start()

In [ ]:
# Stop animation cleanly (call when done)
stop_event.set()
anim_thread.join(timeout=2.0)

In [ ]:
# Cleanup: close all canvases and release Qt resources
for c in [canvas_position, canvas_acceleration, canvas_sonify, canvas_density, canvas_polyline]:
    c.close()
app.quit()